# 🛡️ LOGISTIC REGRESSION WITH REGULARIZATION

## WHAT IS REGULARIZATION?
- IN REAL-WORLD MACHINE LEARNING, MODELS DO NOT FAIL BECAUSE THEY ARE TOO SIMPLE, THEY FAIL BECAUSE THEY **LEARN TOO MUCH**.
- THIS PHENOMENON IS CALLED **OVERFITTING**.
- REGULARIZATION IS A TECHNIQUE USED TO PREVENT **OVERFITTING** BY PENALIZING LARGE WEIGHTS ($\theta$).
- IT ADDS A `"COMPLEXITY TERM"` TO THE COST FUNCTION, FORCING THE MODEL TO KEEP PARAMETERS SMALL AND SIMPLE.

> OVERFITTING OCCURS WHEN A MODEL:
- MEMORIZES TRAINING DATA
- LEARNS NOISE INSTEAD OF PATTERNS
- PERFORMS WELL ON TRAIN DATA BUT FAILS ON UNSEEN DATA

> REGULARIZATION IS A **CONTROL MECHANISM** THAT:
- PENALIZES LARGE WEIGHTS
- LIMITS MODEL COMPLEXITY
- IMPROVES GENERALIZATION
- INTRODUCES BIAS TO REDUCE VARIANCE

LOGISTIC REGRESSION LEARNS PARAMETERS BY MINIMIZING A COST FUNCTION.

WITHOUT REGULARIZATION:

## REGULARIZATION IN LOGISTIC REGRESSION

COST FUNCTION WITHOUT REGULARIZATION 

    J(θ) = -(1/m) Σ [y·log(hθ(x)) + (1 − y)·log(1 − hθ(x))]

### THE THREE TYPES WE WILL IMPLEMENT:

> **1. L2 REGULARIZATION (RIDGE)**
> * **PENALTY:** SQUARED MAGNITUDE OF WEIGHTS ($\theta_j^2$).
> * **EFFECT:** SHRINKS ALL COEFFICIENTS TOWARDS ZERO EVENLY. GOOD FOR COLLINEARITY. SMOOTH DECISION BOUNDARY
> * **MATH:**
>   $$J(\theta)_{reg} = J(\theta) + \frac{\lambda}{2m} \sum_{j=1}^{n} \theta_j^2$$

> **2. L1 REGULARIZATION (LASSO)**
> * **PENALTY:** ABSOLUTE VALUE OF WEIGHTS ($|\theta_j|$).
> * **EFFECT:** CAN SHRINK COEFFICIENTS TO EXACTLY ZERO. PERFORMS **FEATURE SELECTION**. CREATES SPARSE MODELS.
> * **MATH:**
>   $$J(\theta)_{reg} = J(\theta) + \frac{\lambda}{m} \sum_{j=1}^{n} |\theta_j|$$

> **3. ELASTIC NET**
> * **PENALTY:** A COMBINATION OF BOTH L1 AND L2.
> * **EFFECT:** BALANCES SPARSITY (L1) AND STABILITY (L2).
> * **MATH:**
>   $$J(\theta)_{reg} = J(\theta) + r \cdot (\text{L1}) + (1-r) \cdot (\text{L2})$$

---
`NOTE`: WE DO NOT REGULARIZE THE BIAS TERM ($\theta_0$). REGULARIZATION IS APPLIED **ONLY TO WEIGHTS.**

## IMPORTING LIBRARIES
WE WILL USE ONLY NUMPY, PANDAS, AND MATPLOTLIB/SEABORN.

In [1]:
# IMPORT NUMERICAL COMPUTATION LIBRARY
import numpy as np

# IMPORT DATA HANDLING LIBRARY
import pandas as pd

# IMPORT VISUALIZATION LIBRARIES
import matplotlib.pyplot as plt
import seaborn as sns

## VISUALIZATION STYLE

In [2]:
# SETTING UP THE VISUALIZATION STYLE GLOBALLY TO MATCH YOUR REQUIRED FORMAT
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['figure.dpi'] = 500
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'

## LOADING DATA
READING THE DATA FROM CSV FILES.

In [3]:
# LOADING DATA
# IF HEADERS EXIST, REMOVE 'header=None'.

# LOAD X DATA FROM CSV FILE
X_raw = pd.read_csv('DATA/logisticX.csv', header=None).values

# LOAD Y DATA FROM CSV FILE
y_raw = pd.read_csv('DATA/logisticY.csv', header=None).values

In [4]:
# RESHAPING Y TO ENSURE IT IS A COLUMN VECTOR (M, 1)
y_data = y_raw.reshape(-1, 1)

# CHECKING SHAPE
print(f"X SHAPE --> {X_raw.shape}")
print(f"Y SHAPE --> {y_data.shape}")

X SHAPE --> (100, 2)
Y SHAPE --> (100, 1)


## PREPROCESSING DATA
WE WILL NORMALIZE THE DATA AND ADD AN INTERCEPT TERM (BIAS) TO X.

In [5]:

# NORMALIZATION (MIN-MAX SCALING)
# THIS IS CRUCIAL FOR GRADIENT DESCENT CONVERGENCE
# MATH: X_norm = (X - min) / (max - min)

X_min = np.min(X_raw, axis = 0)
X_max = np.max(X_raw, axis = 0)
X_norm = (X_raw - X_min) / (X_max - X_min)

In [6]:
# ADDING INTERCEPT TERM (COLUMN OF ONES) TO X
# MATH: X = [1, x1, x2, ...]

m = len(y_data)
X_data = np.hstack((np.ones((m, 1)), X_norm))

print(f"PROCESSED X SHAPE: {X_data.shape}")

PROCESSED X SHAPE: (100, 3)
